In [1]:
from visibility_graph import VisibilityGraph
import matplotlib.pyplot as plt
import mplleaflet

filename = "maps/kozlovo_extended.osm.pbf"
bbox = [35.7, 56.5, 36.9, 56.6]
map_data = VisibilityGraph(filename, bbox)

/home/denis/.local/lib/python3.8/site-packages/geopandas/_compat.py:84: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.8.1-CAPI-1.13.3). Conversions between both will be slow.
  warnings.warn(


In [8]:
epsilon = 0.003
bbox_comp = 10
map_data.build_dataframe(epsilon=epsilon, bbox_comp=bbox_comp, ellipse=False)

In [9]:
%%time
from find_pair import *
G, fig = map_data.build_graph(find_pair_cutoff, find_pair_brute_force, edges_inside=False, plot=True)
print('edges: ', G.number_of_edges())
print('nodes: ', G.number_of_nodes())
mplleaflet.display(fig=fig)

edges:  410
nodes:  347
CPU times: user 9.97 s, sys: 14.6 ms, total: 9.99 s
Wall time: 10.6 s


/home/denis/anaconda3/lib/python3.8/site-packages/IPython/core/display.py:717: UserWarning: Consider using IPython.display.IFrame instead
  warnings.warn("Consider using IPython.display.IFrame instead")


___
TODO
- inside poly view_angle comparison
- numpy speedup